## Setup environment

In [ ]:
!python -c "import monai" || pip install -q "monai-weekly[nibabel]"
!python -c "import matplotlib" || pip install -q matplotlib
!pip install -q pytorch-lightning~=2.0
%matplotlib inline

## Setup imports

In [1]:
import pytorch_lightning
from pytorch_lightning.callbacks import ModelCheckpoint
from monai.utils import set_determinism
from monai.transforms import (
AsDiscrete,
EnsureChannelFirstd,
Compose,
CropForegroundd,
LoadImaged,
Orientationd,
RandCropByPosNegLabeld,
ScaleIntensityRanged,
Spacingd,
EnsureType,
EnsureTyped,
Resized,
RandAdjustContrastd, 
RandFlipd, 
RandAffined, 
RandAdjustContrastd

)
from monai.networks.nets import UNet, UNETR
from monai.networks.layers import Norm
from monai.metrics import DiceMetric
from monai.losses import DiceLoss
from monai.inferers import sliding_window_inference
from monai.data import CacheDataset, list_data_collate, decollate_batch, DataLoader
from monai.config import print_config
from monai.apps import download_and_extract
import torch
import matplotlib.pyplot as plt
import tempfile
import shutil
import os
import glob
import nibabel as nib
import numpy as np
from natsort import natsorted
from sklearn.model_selection import KFold
from sklearn.metrics import jaccard_score
from neptune.utils import stringify_unsupported

from dotenv import dotenv_values


print_config()

MONAI version: 1.3.2
Numpy version: 1.26.4
Pytorch version: 2.3.1+cu121
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: 59a7211070538586369afd4a01eca0a7fe2e742e
MONAI __file__: /home/<username>/Documents/RISA/3D_segmentation/.venv/lib/python3.10/site-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
ITK version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: 5.2.1
scikit-image version: 0.24.0
scipy version: 1.14.0
Pillow version: 10.3.0
Tensorboard version: 2.17.0
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: NOT INSTALLED or UNKNOWN VERSION.
tqdm version: 4.66.4
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 6.0.0
pandas version: 2.2.2
einops version: 0.8.0
transformers version: NOT INSTALLED or UNKNOWN VERSION.
mlflow version: NOT INSTALLED or UNKNOWN VERSION.
pynrrd version: NOT INSTALLED or UNKNOWN VERSION.
clearml version: NOT INSTALLED or

## Setup data directory

You can specify a directory with the `MONAI_DATA_DIRECTORY` environment variable.  
This allows you to save results and reuse downloads.  
If not specified a temporary directory will be used.

In [2]:
directory = os.environ.get("MONAI_DATA_DIRECTORY")
root_dir = tempfile.mkdtemp() if directory is None else directory
print(root_dir)

/tmp/tmpdkfmk8ll


In [3]:
data_dir = os.path.join(os.getcwd(), 'AeroPath')

## Download dataset

Downloads and extracts the dataset

In [6]:
resource = "https://zenodo.org/records/10069289/files/AeroPath.zip?download=1"
md5 = "3fd5106c175c85d60eaece220f5dfd87"

compressed_file = os.path.join(root_dir, "AeroPath.zip")
if not os.path.exists(data_dir):
    download_and_extract(resource, compressed_file, root_dir, md5)

In [4]:
# segment_name = "left_upper"
# segment_name = "right_upper"
# segment_name = "right_bottom"
# segment_name = "left_bottom"
segment_name = "whole"
# segment_name = "2Q"

## Neptune logger

In [27]:
keys = dotenv_values(".env")
api_key = keys["API_KEY"]

neptune_logger = pytorch_lightning.loggers.NeptuneLogger(
    project="aeropath-workspace/airways-model",
    api_key=api_key,
    tags=['aeropath', 'airways', 'monai', segment_name],
    name='airways-training'
)

In [28]:
checkpoint_callback = ModelCheckpoint(
    monitor='val_dice',
    dirpath=os.path.join(data_dir, 'checkpoints'),  # Directory to save checkpoints
    filename=segment_name + '_{epoch:02d}-{val_dice:.4f}',  # Filename prefix for saving checkpoints
    save_top_k=1,  # Save only the best checkpoint
    mode='max',  # `min` for minimizing the metric, `max` for maximizing
    verbose=True,  # Log a message when saving the best checkpoint
)

## Define the LightningModule

The LightningModule contains a refactoring of your training code. The following module is a refactoring of the code in `spleen_segmentation_3d.ipynb`:

In [31]:
parameters = {
    'spatial_dims': 3,
    'in_channels': 1,
    'out_channels': 2,
    'channels': (16, 32, 64, 128, 256),
    'strides': (2, 2, 2, 2),
    'num_res_units': 2,
    'norm': Norm.BATCH
}


class UNetClass(pytorch_lightning.LightningModule):
    def __init__(self, mode, roi_size, spatial_size):
        super().__init__()
        self._model = UNet(**parameters)
        self.loss_function = DiceLoss(to_onehot_y=True, softmax=True)
        self.post_pred = Compose([EnsureType("tensor", device="cpu"), AsDiscrete(argmax=True, to_onehot=2)])
        self.post_label = Compose([EnsureType("tensor", device="cpu"), AsDiscrete(to_onehot=2)])
        self.dice_metric = DiceMetric(include_background=False, reduction="mean", get_not_nans=False)
        self.best_val_dice = 0
        self.best_val_epoch = 0
        self.validation_step_outputs = []

        self.mode = mode
        self.roi_size = roi_size
        self.spatial_size = spatial_size

    def forward(self, x):
        return self._model(x)

    def prepare_data(self, prepare_val_data=True, prepare_test_data=True):
        # # set up the correct data path
        def get_patterns(mode):
            patterns = {
                'whole': ('AeroPath/**/*_CT_HR_label_airways.nii.gz', 'AeroPath/**/*_CT_HR.nii.gz'),
                '1Q': ('nonoverlapping_labels/**/quadrant_1_*.nii.gz', 'nonoverlapping_quadrants/**/quadrant_1_*_CT_HR.nii.gz'),
                '2Q': ('nonoverlapping_labels/**/quadrant_2_*.nii.gz', 'nonoverlapping_quadrants/**/quadrant_2_*_CT_HR.nii.gz'),
                'left_bottom': ('dataset/airways_patched_4/**/*left_bottom_*.nii.gz', 'dataset/scan_patched_4/**/*left_bottom_*.nii.gz'),
                'left_upper': ('dataset/airways_patched_4/**/*left_upper_*.nii.gz', 'dataset/scan_patched_4/**/*left_upper_*.nii.gz'),
                'right_bottom': ('dataset/airways_patched_4/**/*right_bottom_*.nii.gz', 'dataset/scan_patched_4/**/*right_bottom_*.nii.gz'),
                'right_upper': ('dataset/airways_patched_4/**/*right_upper_*.nii.gz', 'dataset/scan_patched_4/**/*right_upper_*.nii.gz')
            }
            return patterns.get(mode, (None, None))

        pattern_labels, pattern_images = get_patterns(self.mode)
        if pattern_labels and pattern_images:
            train_labels = sorted(glob.glob(pattern_labels, recursive=True))
            train_images = sorted(glob.glob(pattern_images, recursive=True))

        data_dicts = [
            {"image": image_name, "label": label_name} for image_name, label_name in zip(train_images, train_labels)
        ]
        train_files, val_files = data_dicts[:-9], data_dicts[-9:]

        # set deterministic training for reproducibility
        set_determinism(seed=0)

        # define the data transforms
        train_transforms = Compose(
            [
                LoadImaged(keys=["image", "label"]),
                EnsureChannelFirstd(keys=["image", "label"]),
                Orientationd(keys=["image", "label"], axcodes="RAS"),
                Spacingd(
                    keys=["image", "label"],
                    # pixdim=(1.5, 1.5, 2.0),
                    pixdim=(1.0, 1.0, 1.35),
                    mode=("bilinear", "nearest"),
                ),
                ScaleIntensityRanged(
                    keys=["image"],
                    a_min=-1024,
                    a_max=1024,
                    b_min=0.0,
                    b_max=1.0,
                    clip=True,
                ),
                CropForegroundd(keys=["image", "label"], source_key="image"),
                # Resized(keys=["image", "label"], spatial_size=self.spatial_size),
                # randomly crop out patch samples from
                # big image based on pos / neg ratio
                # the image centers of negative samples
                # must be in valid image area
                RandCropByPosNegLabeld(
                    keys=["image", "label"],
                    label_key="label",
                    spatial_size=(64, 64, 64),
                    pos=1,
                    neg=1,
                    num_samples=4,
                    image_key="image",
                    image_threshold=0,
                ),

                # user can also add other random transforms
                #                 RandAffined(
                #                     keys=['image', 'label'],
                #                     mode=('bilinear', 'nearest'),
                #                     prob=1.0,
                #                     spatial_size=(96, 96, 96),
                #                     rotate_range=(0, 0, np.pi/15),
                #                     scale_range=(0.1, 0.1, 0.1)),
                # Adding the data augmentation transforms with a probability of 50%
                # RandFlipd(
                #     keys=["image", "label"],
                #     spatial_axis=[0],  # Horizontal flip
                #     prob=0.5
                # ),
                # RandFlipd(
                #     keys=["image", "label"],
                #     spatial_axis=[1],  # Vertical flip
                #     prob=0.5
                # ),
                # RandAffined(
                #     keys=["image", "label"],
                #     prob=0.5,
                #     rotate_range=(np.deg2rad(20), np.deg2rad(20), np.deg2rad(20)),
                #     translate_range=(0.2, 0.2, 0.2),
                #     scale_range=(0.5, 1.5),
                #     mode=('bilinear', 'nearest')
                # ),
                RandAdjustContrastd(
                    keys=["image"],
                    gamma=(0.5, 2.0),
                    prob=0.0
                ),
                EnsureTyped(keys=["image", "label"]),
            ]
        )
        val_transforms = Compose(
            [
                LoadImaged(keys=["image", "label"]),
                EnsureChannelFirstd(keys=["image", "label"]),
                Orientationd(keys=["image", "label"], axcodes="RAS"),
                Spacingd(
                    keys=["image", "label"],
                    # pixdim=(1.5, 1.5, 2.0),
                    pixdim=(1.0, 1.0, 1.35),
                    mode=("bilinear", "nearest"),
                ),
                ScaleIntensityRanged(
                    keys=["image"],
                    a_min=-1024,
                    a_max=1024,
                    b_min=0.0,
                    b_max=1.0,
                    clip=True,
                ),
                # Resized(keys=["image", "label"], spatial_size=self.spatial_size),
                CropForegroundd(keys=["image", "label"], source_key="image"),
            ]
        )
                    

        # we use cached datasets - these are 10x faster than regular datasets
        if prepare_test_data:
            self.train_ds = CacheDataset(
                data=train_files,
                transform=train_transforms,
                cache_rate=1.0,
                num_workers=4,
            )
        if prepare_val_data:
            self.val_ds = CacheDataset(
                data=val_files,
                transform=val_transforms,
                cache_rate=1.0,
                num_workers=4,
            )


    def train_dataloader(self):
        train_loader = DataLoader(
            self.train_ds,
            batch_size=4,
            shuffle=True,
            num_workers=4,
            collate_fn=list_data_collate,
        )
        return train_loader

    def val_dataloader(self):
        val_loader = DataLoader(self.val_ds, batch_size=1, num_workers=4)
        return val_loader

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self._model.parameters(), 1e-4)
        return optimizer

    def training_step(self, batch, batch_idx):
        images, labels = batch["image"], batch["label"]
        output = self.forward(images)
        loss = self.loss_function(output, labels)
        tensorboard_logs = {"train_loss": loss.item()}
        self.log('train_loss', loss, on_step=False, on_epoch=True, prog_bar=False, logger=True)
        return {"loss": loss, "log": tensorboard_logs}

    
    def validation_step(self, batch, batch_idx):
        images, labels = batch["image"], batch["label"]
        roi_size = self.roi_size
        sw_batch_size = 4
        outputs = sliding_window_inference(images, roi_size, sw_batch_size, self)
        loss = self.loss_function(outputs, labels)
        outputs = [self.post_pred(i) for i in decollate_batch(outputs)]
        labels = [self.post_label(i) for i in decollate_batch(labels)]
        self.dice_metric(y_pred=outputs, y=labels)

        outputs_np = [output.argmax(dim=0).cpu().numpy() for output in outputs]
        labels_np = [label.argmax(dim=0).cpu().numpy() for label in labels]
        iou = np.mean([jaccard_score(ln.flatten(), on.flatten(), average='macro') for ln, on in zip(labels_np, outputs_np)])

        d = {"val_loss": loss, "val_number": len(outputs), "iou": iou}
        self.validation_step_outputs.append(d)
        return d
    

    
    def perform_inference(self, model, data):
        # Perform inference using the model
        with torch.no_grad():
            data = torch.DoubleTensor(data)  # Convert data to type Double
            model_output = model(data.unsqueeze(0))
        return model_output

    def on_validation_epoch_end(self):
        val_loss, num_items, total_iou = 0, 0, 0
        for output in self.validation_step_outputs:
            val_loss += output["val_loss"].sum().item()
            num_items += output["val_number"]
            total_iou += output["iou"] * output["val_number"]
        mean_val_dice = self.dice_metric.aggregate().item()
        mean_val_iou = total_iou / num_items
        self.dice_metric.reset()
        mean_val_loss = torch.tensor(val_loss / num_items)
        tensorboard_logs = {
            "val_dice": mean_val_dice,
            "val_loss": mean_val_loss,
            "val_iou": mean_val_iou,
        }
        if mean_val_dice > self.best_val_dice:
            self.best_val_dice = mean_val_dice
            self.best_val_epoch = self.current_epoch
        print(
            f"current epoch: {self.current_epoch} "
            f"current mean dice: {mean_val_dice:.4f} "
            f"current mean iou: {mean_val_iou:.4f}"
            f"\nbest mean dice: {self.best_val_dice:.4f} "
            f"at epoch: {self.best_val_epoch}"
        )
        self.validation_step_outputs.clear()  # free memory
        self.log('val_dice', mean_val_dice, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        self.log('val_loss', mean_val_loss, on_step=False, on_epoch=True, prog_bar=False, logger=True)
        self.log('val_iou', mean_val_iou, on_step=False, on_epoch=True, prog_bar=False, logger=True)

        return {"log": tensorboard_logs}
    
    
    def dice_score(self, prediction_tensor, label_tensor):
        # Compute Dice score
        dice_metric = DiceMetric(include_background=True, reduction="mean")
        dice_metric(y_pred=prediction_tensor, y=label_tensor)
        dice_score = dice_metric.aggregate().item()
        dice_metric.reset()

        print(dice_score)

In [32]:
Net_segment       = UNetClass(mode = segment_name,       roi_size=(160*2, 160, 160),   spatial_size=(160*2, 160, 160))
NetWhole          = UNetClass(mode = 'whole',            roi_size=(128, 128, 144),   spatial_size=(128, 128, 144))
Net1Q             = UNetClass(mode = '1Q',               roi_size=(160*2, 160, 160), spatial_size=(160*2, 160, 160))
Net2Q             = UNetClass(mode = '2Q',               roi_size=(160*2, 160, 160), spatial_size=(160*2, 160, 160))

# NetWhole    = UNetClass(mode = 'whole', roi_size=(192, 192, 212),   spatial_size=(192, 192, 212))

## Log model parameters

In [33]:
neptune_logger.experiment["model/parameters"] = stringify_unsupported(parameters)

[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/aeropath-workspace/airways-model/e/AIR-89


## Run the training

In [34]:
# initialise the LightningModule
# net = Net1Q
# net = Net2Q
# net = Net_segment
net = NetWhole
# set up loggers and checkpoints
log_dir = os.path.join(root_dir, "logs")

# initialise Lightning's trainer.
trainer = pytorch_lightning.Trainer(
    devices=[0],
    max_epochs=600,
    logger=neptune_logger,
    enable_checkpointing=True,
    callbacks=[checkpoint_callback],
    num_sanity_val_steps=1,
    log_every_n_steps=16,
)
# train
trainer.fit(net)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/pawel/Documents/RISA/3D_segmentation/.venv/lib/python3.10/site-packages/monai/utils/deprecate_utils.py:321: FutureWarning: monai.transforms.croppad.dictionary CropForegroundd.__init__:allow_smaller: Current default value of argument `allow_smaller=True` has been deprecated since version 1.2. It will be changed to `allow_smaller=False` in version 1.5.
  warn_deprecated(argname, msg, warning_category)
Loading dataset: 100%|██████████| 9/9 [00:37<00:00,  4.19s/it]
/home/pawel/Documents/RISA/3D_segmentation/.venv/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:652: Checkpoint directory /home/pawel/Documents/RISA/3D_segmentation/AeroPath/checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type     | Params | Mode 
---------------------------------------------------
0 | _model        | UNet     |

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

current epoch: 0 current mean dice: 0.0024 current mean iou: 0.3880
best mean dice: 0.0024 at epoch: 0


/home/pawel/Documents/RISA/3D_segmentation/.venv/lib/python3.10/site-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (5) is smaller than the logging interval Trainer(log_every_n_steps=16). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 429, global step 2150: 'val_dice' was not in top 1


current epoch: 429 current mean dice: 0.6561 current mean iou: 0.7484
best mean dice: 0.7416 at epoch: 372
[neptune] [error  ] Error occurred during asynchronous operation processing: X-coordinates (step) must be strictly increasing for series attribute: training/epoch. Invalid point: 2149.0


Validation: |          | 0/? [00:00<?, ?it/s]

/home/pawel/Documents/RISA/3D_segmentation/.venv/lib/python3.10/site-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


In [ ]:
print(f"train completed, best_metric: {net.best_val_dice:.4f} " f"at epoch {net.best_val_epoch}")

In [ ]:
import torch
from monai.networks.nets import UNet


# Load the model weights from the checkpoint file
checkpoint_path = 'best-checkpoint.ckpt'
model = Net1Q.load_from_checkpoint('1Q_clipped_resized128_128_144_roibest_metric: 0.2797 at epoch 495.ckpt', mode = '1Q', roi_size=(160*2, 160, 160), spatial_size=(160*2, 160, 160))

# Set the model to evaluation mode
model.eval()


## Model Ensembling

## View training in tensorboard

Please uncomment the following cell to load tensorboard results.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=$log_dir

## Load model and create prediction files

In [ ]:
netLeftUpper = UNetClass.load_from_checkpoint('AeroPath/checkpoints/left_upper_epoch=592-val_dice=0.7192.ckpt',
                                    mode = 'left_upper',
                                    roi_size=(160*2, 160, 160),
                                    spatial_size=(160*2, 160, 160))
netLeftBottom = UNetClass.load_from_checkpoint('AeroPath/checkpoints/left_bottom_epoch=525-val_dice=0.4427.ckpt',
                                    mode = 'left_bottom',
                                    roi_size=(160*2, 160, 160),
                                    spatial_size=(160*2, 160, 160))
netRightUpper = UNetClass.load_from_checkpoint('AeroPath/checkpoints/right_upper_epoch=449-val_dice=0.7599.ckpt',
                                    mode = 'right_upper',
                                    roi_size=(160*2, 160, 160),
                                    spatial_size=(160*2, 160, 160))
netRightBottom = UNetClass.load_from_checkpoint('AeroPath/checkpoints/right_bottom_epoch=482-val_dice=0.5261.ckpt',
                                    mode = 'right_bottom',
                                    roi_size=(160*2, 160, 160),
                                    spatial_size=(160*2, 160, 160))
netWhole = UNetClass.load_from_checkpoint('AeroPath/checkpoints/whole_lower_pixdim.ckpt',
                                    mode = 'whole',
                                    roi_size=(128, 128, 144),
                                    spatial_size=(128, 128, 144))

netLeftUpper.prepare_data(prepare_test_data=False)
netLeftBottom.prepare_data(prepare_test_data=False)
netRightBottom.prepare_data(prepare_test_data=False)
netRightUpper.prepare_data(prepare_test_data=False)
netWhole.prepare_data(prepare_test_data=False)

nets = [netLeftUpper, netLeftBottom, netRightUpper, netRightBottom, netWhole]

['dataset/scan_patched_4/left_upper/0_left_upper_CT_HR.nii.gz', 'dataset/scan_patched_4/left_upper/10_left_upper_CT_HR.nii.gz', 'dataset/scan_patched_4/left_upper/11_left_upper_CT_HR.nii.gz', 'dataset/scan_patched_4/left_upper/12_left_upper_CT_HR.nii.gz', 'dataset/scan_patched_4/left_upper/13_left_upper_CT_HR.nii.gz', 'dataset/scan_patched_4/left_upper/14_left_upper_CT_HR.nii.gz', 'dataset/scan_patched_4/left_upper/15_left_upper_CT_HR.nii.gz', 'dataset/scan_patched_4/left_upper/16_left_upper_CT_HR.nii.gz', 'dataset/scan_patched_4/left_upper/17_left_upper_CT_HR.nii.gz', 'dataset/scan_patched_4/left_upper/18_left_upper_CT_HR.nii.gz', 'dataset/scan_patched_4/left_upper/19_left_upper_CT_HR.nii.gz', 'dataset/scan_patched_4/left_upper/1_left_upper_CT_HR.nii.gz', 'dataset/scan_patched_4/left_upper/20_left_upper_CT_HR.nii.gz', 'dataset/scan_patched_4/left_upper/21_left_upper_CT_HR.nii.gz', 'dataset/scan_patched_4/left_upper/22_left_upper_CT_HR.nii.gz', 'dataset/scan_patched_4/left_upper/23_lef

Loading dataset: 100%|██████████| 9/9 [00:10<00:00,  1.14s/it]


['dataset/scan_patched_4/left_bottom/0_left_bottom_CT_HR.nii.gz', 'dataset/scan_patched_4/left_bottom/10_left_bottom_CT_HR.nii.gz', 'dataset/scan_patched_4/left_bottom/11_left_bottom_CT_HR.nii.gz', 'dataset/scan_patched_4/left_bottom/12_left_bottom_CT_HR.nii.gz', 'dataset/scan_patched_4/left_bottom/13_left_bottom_CT_HR.nii.gz', 'dataset/scan_patched_4/left_bottom/14_left_bottom_CT_HR.nii.gz', 'dataset/scan_patched_4/left_bottom/15_left_bottom_CT_HR.nii.gz', 'dataset/scan_patched_4/left_bottom/16_left_bottom_CT_HR.nii.gz', 'dataset/scan_patched_4/left_bottom/17_left_bottom_CT_HR.nii.gz', 'dataset/scan_patched_4/left_bottom/18_left_bottom_CT_HR.nii.gz', 'dataset/scan_patched_4/left_bottom/19_left_bottom_CT_HR.nii.gz', 'dataset/scan_patched_4/left_bottom/1_left_bottom_CT_HR.nii.gz', 'dataset/scan_patched_4/left_bottom/20_left_bottom_CT_HR.nii.gz', 'dataset/scan_patched_4/left_bottom/21_left_bottom_CT_HR.nii.gz', 'dataset/scan_patched_4/left_bottom/22_left_bottom_CT_HR.nii.gz', 'dataset/sc

Loading dataset: 100%|██████████| 9/9 [00:05<00:00,  1.69it/s]


['dataset/scan_patched_4/right_bottom/0_right_bottom_CT_HR.nii.gz', 'dataset/scan_patched_4/right_bottom/10_right_bottom_CT_HR.nii.gz', 'dataset/scan_patched_4/right_bottom/11_right_bottom_CT_HR.nii.gz', 'dataset/scan_patched_4/right_bottom/12_right_bottom_CT_HR.nii.gz', 'dataset/scan_patched_4/right_bottom/13_right_bottom_CT_HR.nii.gz', 'dataset/scan_patched_4/right_bottom/14_right_bottom_CT_HR.nii.gz', 'dataset/scan_patched_4/right_bottom/15_right_bottom_CT_HR.nii.gz', 'dataset/scan_patched_4/right_bottom/16_right_bottom_CT_HR.nii.gz', 'dataset/scan_patched_4/right_bottom/17_right_bottom_CT_HR.nii.gz', 'dataset/scan_patched_4/right_bottom/18_right_bottom_CT_HR.nii.gz', 'dataset/scan_patched_4/right_bottom/19_right_bottom_CT_HR.nii.gz', 'dataset/scan_patched_4/right_bottom/1_right_bottom_CT_HR.nii.gz', 'dataset/scan_patched_4/right_bottom/20_right_bottom_CT_HR.nii.gz', 'dataset/scan_patched_4/right_bottom/21_right_bottom_CT_HR.nii.gz', 'dataset/scan_patched_4/right_bottom/22_right_bot

Loading dataset: 100%|██████████| 9/9 [00:05<00:00,  1.72it/s]


['dataset/scan_patched_4/right_upper/0_right_upper_CT_HR.nii.gz', 'dataset/scan_patched_4/right_upper/10_right_upper_CT_HR.nii.gz', 'dataset/scan_patched_4/right_upper/11_right_upper_CT_HR.nii.gz', 'dataset/scan_patched_4/right_upper/12_right_upper_CT_HR.nii.gz', 'dataset/scan_patched_4/right_upper/13_right_upper_CT_HR.nii.gz', 'dataset/scan_patched_4/right_upper/14_right_upper_CT_HR.nii.gz', 'dataset/scan_patched_4/right_upper/15_right_upper_CT_HR.nii.gz', 'dataset/scan_patched_4/right_upper/16_right_upper_CT_HR.nii.gz', 'dataset/scan_patched_4/right_upper/17_right_upper_CT_HR.nii.gz', 'dataset/scan_patched_4/right_upper/18_right_upper_CT_HR.nii.gz', 'dataset/scan_patched_4/right_upper/19_right_upper_CT_HR.nii.gz', 'dataset/scan_patched_4/right_upper/1_right_upper_CT_HR.nii.gz', 'dataset/scan_patched_4/right_upper/20_right_upper_CT_HR.nii.gz', 'dataset/scan_patched_4/right_upper/21_right_upper_CT_HR.nii.gz', 'dataset/scan_patched_4/right_upper/22_right_upper_CT_HR.nii.gz', 'dataset/sc

Loading dataset: 100%|██████████| 9/9 [00:05<00:00,  1.72it/s]


['AeroPath/1/1_CT_HR.nii.gz', 'AeroPath/10/10_CT_HR.nii.gz', 'AeroPath/11/11_CT_HR.nii.gz', 'AeroPath/12/12_CT_HR.nii.gz', 'AeroPath/13/13_CT_HR.nii.gz', 'AeroPath/14/14_CT_HR.nii.gz', 'AeroPath/15/15_CT_HR.nii.gz', 'AeroPath/16/16_CT_HR.nii.gz', 'AeroPath/17/17_CT_HR.nii.gz', 'AeroPath/18/18_CT_HR.nii.gz', 'AeroPath/19/19_CT_HR.nii.gz', 'AeroPath/2/2_CT_HR.nii.gz', 'AeroPath/20/20_CT_HR.nii.gz', 'AeroPath/21/21_CT_HR.nii.gz', 'AeroPath/22/22_CT_HR.nii.gz', 'AeroPath/23/23_CT_HR.nii.gz', 'AeroPath/24/24_CT_HR.nii.gz', 'AeroPath/25/25_CT_HR.nii.gz', 'AeroPath/26/26_CT_HR.nii.gz', 'AeroPath/27/27_CT_HR.nii.gz', 'AeroPath/3/3_CT_HR.nii.gz', 'AeroPath/4/4_CT_HR.nii.gz', 'AeroPath/5/5_CT_HR.nii.gz', 'AeroPath/6/6_CT_HR.nii.gz', 'AeroPath/7/7_CT_HR.nii.gz', 'AeroPath/8/8_CT_HR.nii.gz', 'AeroPath/9/9_CT_HR.nii.gz', 'dataset/airways_patched_4/left_bottom/0_left_bottom_CT_HR.nii.gz', 'dataset/airways_patched_4/left_bottom/10_left_bottom_CT_HR.nii.gz', 'dataset/airways_patched_4/left_bottom/11_l

Loading dataset: 100%|██████████| 9/9 [00:20<00:00,  2.33s/it]


In [ ]:
# test validation dataset labels
for i, val_data in enumerate(net.val_dataloader()):
    label = val_data['label'].cpu().numpy()[0, 0, :, :, :]
    nib.save(nib.Nifti1Image(label.astype(float), nib.load('AeroPath/1/1_CT_HR_label_airways.nii.gz').affine), f'labels_spacingd_10less/{i}.nii.gz')

In [ ]:
def create_directory(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [ ]:
for net in nets:
    net.eval()
    device = torch.device("cuda:0")
    net.to(device)
    with torch.no_grad():
        for i, val_data in enumerate(net.val_dataloader()):
            roi_size = net.roi_size
            sw_batch_size = 4
            val_outputs = sliding_window_inference(val_data["image"].to(device), roi_size, sw_batch_size, net)
            pred = torch.argmax(val_outputs, dim=1).detach().cpu()
            plt.show()

            pred_np = pred.cpu().numpy()[0, :, :, :]

            dir_name = net.mode

            create_directory(f'pred/{dir_name}')
            nib.save(nib.Nifti1Image(pred_np.astype(float), val_data["image"].affine[0]), f'pred/{dir_name}/{i}.nii.gz')


## Interpolate, Concat and ensemble predictions

In [ ]:
import numpy as np
import torch
import torch.nn.functional as F
import nibabel as nib
from monai.transforms import Compose, AsDiscrete
from monai.metrics import DiceMetric

def interpolate_predictions(predictions, target_shape):
    # Ensure predictions tensor has 5 dimensions: (N, C, D, H, W)
    predictions = torch.tensor(predictions).unsqueeze(0).unsqueeze(0)  # Add batch and channel dimensions

    # Interpolate the predictions using nearest neighbor interpolation
    interpolated_predictions = F.interpolate(predictions, size=target_shape, mode='nearest')
    
    # Remove batch and channel dimensions after interpolation
    interpolated_predictions = interpolated_predictions.squeeze(0).squeeze(0).numpy()
    
    return interpolated_predictions

# Function to calculate Dice score
def calc_dice_score(pred, label):
    post_pred = Compose([AsDiscrete(argmax=False, to_onehot=2)])
    post_label = Compose([AsDiscrete(to_onehot=2)])

    # Ensure the inputs have the correct dimensions
    pred = torch.tensor(pred).unsqueeze(0).unsqueeze(0)
    label = torch.tensor(label).unsqueeze(0).unsqueeze(0)

    prediction_tensor = post_pred(pred)
    label_tensor = post_label(label)

    # Compute Dice score
    dice_metric = DiceMetric(include_background=False, reduction="mean")
    dice_metric(y_pred=prediction_tensor, y=label_tensor)

    dice_score = dice_metric.aggregate().item()
    dice_metric.reset()

    return dice_score


labels = natsorted(glob.glob('pred/*/*', recursive=True))

# Example usage with NIfTI files

for idx, _ in enumerate(labels.unique()):
    # label = nib.load(f'labels_resized/{idx}.nii.gz').get_fdata()
    # whole = nib.load(f'pred/whole_resized_roi160/{idx}.nii.gz').get_fdata()
    whole = nib.load(f'pred/whole/{idx}.nii.gz').get_fdata()

    pred_left_bottom = nib.load(f'pred/left_bottom/{idx}.nii.gz').get_fdata()
    pred_left_upper = nib.load(f'pred/left_upper/{idx}.nii.gz').get_fdata()
    pred_right_bottom = nib.load(f'pred/right_bottom/{idx}.nii.gz').get_fdata()
    pred_right_upper = nib.load(f'pred/right_upper/{idx}.nii.gz').get_fdata()

    # Interpolate predictions to allow for concatenation

    inter_pred_left_bottom = interpolate_predictions(pred_left_bottom, (*whole.shape[:-1], pred_left_bottom.shape[-1]))
    inter_pred_left_upper = interpolate_predictions(pred_left_upper, (*whole.shape[:-1], pred_left_upper.shape[-1]))
    inter_pred_right_bottom = interpolate_predictions(pred_right_bottom, (*whole.shape[:-1], pred_right_bottom.shape[-1]))
    inter_pred_right_upper = interpolate_predictions(pred_right_upper, (*whole.shape[:-1], pred_right_upper.shape[-1]))


#     # Merge interpolated predictions along the last axis (z-axis)
    merged_left = np.concatenate((inter_pred_left_bottom, inter_pred_left_upper), axis=2)
    merged_right = np.concatenate((inter_pred_right_bottom, inter_pred_right_upper), axis=2)

    merged = np.concatenate((merged_left, merged_right), axis=1)

    # Interpolate predictions to match whole.shape

    merged = interpolate_predictions(merged, whole.shape)

#     # Save the merged predictions as a NIfTI file
    merged_nifti = nib.Nifti1Image(merged, nib.load('AeroPath/1/1_CT_HR.nii.gz').affine)
    nib.save(merged_nifti, 'pred_test_merged.nii.gz')
    # print(f'Merged shape: {merged.shape}')

    # Perform ensembling using the maximum values
    ensembled = np.maximum(merged, whole)
    
    ensembled[:5, :, :] = 0
    ensembled[-5:, :, :] = 0
    ensembled[:, :5, :] = 0
    ensembled[:, -5:, :] = 0
    ensembled[:, :, :5] = 0
    ensembled[:, :, -5:] = 0

    # Save the ensembled predictions as a NIfTI file
    ensembled_nifti = nib.Nifti1Image(ensembled, nib.load('AeroPath/1/1_CT_HR.nii.gz').affine)
    create_directory('pred/ensembled')
    nib.save(ensembled_nifti, f'pred/ensembled/{idx}.nii.gz')

    # Compute Dice score
    # print(f'whole: {whole.shape}, label: {label.shape}')
    # dice_whole = calc_dice_score(whole, label)
    # print(f'Dice Score for whole prediction: {dice_whole:.4f}')



    # dice_ensembled = calc_dice_score(ensembled, label)
    # print(f'Dice Score for ensembled prediction: {dice_ensembled:.4f}')

    print('*'*50)

# Dice Score for whole prediction: 0.8711
# Dice Score for ensembled prediction: 0.8846
# **************************************************
# Dice Score for whole prediction: 0.7663
# Dice Score for ensembled prediction: 0.7592
# **************************************************
# Dice Score for whole prediction: 0.7318
# Dice Score for ensembled prediction: 0.7432
# **************************************************
# Dice Score for whole prediction: 0.8730
# Dice Score for ensembled prediction: 0.8764
# **************************************************
# Dice Score for whole prediction: 0.8862
# Dice Score for ensembled prediction: 0.8930
# **************************************************
# Dice Score for whole prediction: 0.7021
# Dice Score for ensembled prediction: 0.7184
# **************************************************
# Dice Score for whole prediction: 0.7940
# Dice Score for ensembled prediction: 0.8396
# **************************************************
# Dice Score for whole prediction: 0.6629
# Dice Score for ensembled prediction: 0.6776
# **************************************************
# Dice Score for whole prediction: 0.8969
# Dice Score for ensembled prediction: 0.9105
# **************************************************

AttributeError: 'list' object has no attribute 'unique'

In [ ]:
ensembled_nifti = nib.Nifti1Image(ensembled, nib.load('AeroPath/1/1_CT_HR.nii.gz').affine)
nib.save(ensembled_nifti, f'pred/ensembled/{idx}.nii.gz')

# Compute Dice score
dice_whole = calc_dice_score(whole, label)
print(f'Dice Score for whole prediction: {dice_whole:.4f}')

dice_ensembled = calc_dice_score(ensembled, label)
print(f'Dice Score for ensembled prediction: {dice_ensembled:.4f}')

In [ ]:
def calc_dice_score(pred, label):
    post_pred = Compose([AsDiscrete(argmax=False, to_onehot=2)])
    post_label = Compose([AsDiscrete(to_onehot=2)])

    # Ensure the inputs have the correct dimensions
    pred = torch.tensor(pred).unsqueeze(0).unsqueeze(0)
    label = torch.tensor(label).unsqueeze(0).unsqueeze(0)

    prediction_tensor = post_pred(pred)
    label_tensor = post_label(label)

    # Compute Dice score
    dice_metric = DiceMetric(include_background=False, reduction="mean")
    dice_metric(y_pred=prediction_tensor, y=label_tensor)

    dice_score = dice_metric.aggregate().item()
    dice_metric.reset()

    return dice_score

label = val_data['label'].cpu().numpy()[0, 0, :, :, :]
dice_whole = calc_dice_score(pred, label)
print(f'Dice Score for whole prediction: {dice_whole:.4f}')

## Cleanup data directory

Remove directory if a temporary was used.

In [ ]:
if directory is None:
    shutil.rmtree(root_dir)